In [63]:
itemPath = "association_rules_data/EB-build-goods.sql"
receiptspath = "association_rules_data/75000-out1.csv"

In [64]:
with open(receiptspath,"r") as receiptsfile:
    receiptsdata = receiptsfile.read().split('\n')

In [65]:
#1st col is auto_incr, and after purchased items
receiptsData[:5]

['1, 11, 21',
 '2, 7, 11, 37, 45',
 '3, 3, 33, 42',
 '4, 5, 12, 17, 47',
 '5, 6, 18, 42']

In [66]:
len(receiptsData)

75001

In [67]:
baskets = [line.split(", ")[1:] for line in receiptsdata[0:-1]]

In [68]:
baskets[0:5]

[['11', '21'],
 ['7', '11', '37', '45'],
 ['3', '33', '42'],
 ['5', '12', '17', '47'],
 ['6', '18', '42']]

In [69]:
len(baskets)

75000

In [72]:
with open(itempath,"r") as itemfile:
    lines = itemfile.read().split('\n')

In [73]:
len(lines)

51

In [74]:
lines[:4]

["insert into goods values (0,'Chocolate','Cake',8.95,'Food');",
 "insert into goods values (1,'Lemon','Cake',8.95,'Food');",
 "insert into goods values (2,'Casino','Cake',15.95,'Food');",
 "insert into goods values (3,'Opera','Cake',15.95,'Food');"]

In [75]:
items = [line.split("(")[1][0:-2].split(",") for line in lines[0:-1]]
items[0:5]

[['0', "'Chocolate'", "'Cake'", '8.95', "'Food'"],
 ['1', "'Lemon'", "'Cake'", '8.95', "'Food'"],
 ['2', "'Casino'", "'Cake'", '15.95', "'Food'"],
 ['3', "'Opera'", "'Cake'", '15.95', "'Food'"],
 ['4', "'Strawberry'", "'Cake'", '11.95', "'Food'"]]

In [83]:
itemMap = {line[0]:line[1]+" "+line[2] for line in items}
print(itemMap.get('0'))
itemMap.get('3')

'Chocolate' 'Cake'


"'Opera' 'Cake'"

In [106]:
itemMap

{'0': "'Chocolate' 'Cake'",
 '1': "'Lemon' 'Cake'",
 '10': "'Almond' 'Tart'",
 '11': "'Apple' 'Pie'",
 '12': "'Apple' 'Tart'",
 '13': "'Apricot' 'Tart'",
 '14': "'Berry' 'Tart'",
 '15': "'Blackberry' 'Tart'",
 '16': "'Blueberry' 'Tart'",
 '17': "'Chocolate' 'Tart'",
 '18': "'Cherry' 'Tart'",
 '19': "'Lemon' 'Tart'",
 '2': "'Casino' 'Cake'",
 '20': "'Pecan' 'Tart'",
 '21': "'Ganache' 'Cookie'",
 '22': "'Gongolais' 'Cookie'",
 '23': "'Raspberry' 'Cookie'",
 '24': "'Lemon' 'Cookie'",
 '25': "'Chocolate' 'Meringue'",
 '26': "'Vanilla' 'Meringue'",
 '27': "'Marzipan' 'Cookie'",
 '28': "'Tuile' 'Cookie'",
 '29': "'Walnut' 'Cookie'",
 '3': "'Opera' 'Cake'",
 '30': "'Almond' 'Croissant'",
 '31': "'Apple' 'Croissant'",
 '32': "'Apricot' 'Croissant'",
 '33': "'Cheese' 'Croissant'",
 '34': "'Chocolate' 'Croissant'",
 '35': "'Apricot' 'Danish'",
 '36': "'Apple' 'Danish'",
 '37': "'Almond' 'Twist'",
 '38': "'Almond' 'Bear Claw'",
 '39': "'Blueberry' 'Danish'",
 '4': "'Strawberry' 'Cake'",
 '40': "'

In [77]:
numItems = len(items)
numBaskets = len(baskets)

In [84]:
numItems, numBaskets

(50, 75000)

In [85]:
#basket is transaction here
def support(itemset):
    basketSubset = baskets
    for item in itemset: 
        basketSubset = [basket for basket in basketSubset if item in basket]
    return float(len(basketSubset))/float(numBaskets)

In [86]:
#.28% of all transactions contain both the items 2 and 24
support(['2','24'])

0.00288

* Find 1 item sets ------- support
* Find 2 item sets ------- support
* Generate 2 item rules ---- confidence
* Find 3 item sets ------- support
* Generate 3 item rules ---- confidence

In [87]:
supportItems1 = []
minsupport=0.01
for item in range(numItems):
    itemset=[str(item)]
    if support(itemset)>=minsupport:
        supportItems1.append(str(item))

In [88]:
supportItems1

['0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 '10',
 '11',
 '12',
 '13',
 '14',
 '15',
 '16',
 '17',
 '18',
 '19',
 '20',
 '21',
 '22',
 '23',
 '24',
 '25',
 '26',
 '27',
 '28',
 '29',
 '30',
 '31',
 '32',
 '33',
 '34',
 '35',
 '36',
 '37',
 '38',
 '39',
 '40',
 '41',
 '42',
 '43',
 '44',
 '45',
 '46',
 '47',
 '48',
 '49']

In [89]:
import itertools 

In [93]:
a = [1, 2, 3, 4]
for it in itertools.combinations(a, 3):
    print(it)

(1, 2, 3)
(1, 2, 4)
(1, 3, 4)
(2, 3, 4)


In [90]:
# i -- size of item sets in this iteration
# supportItems -- The items from which the sets are generated
# assocRules -- Rules generated till now -- given from confidence
# newSupportItems -- Items left over after this iteration

def aprioriIteration(i,supportItems,assocRules,newSupportItems,minsupport,minconfidence):
    
    for itemset in itertools.combinations(supportItems,i): 
        itemset = list(itemset)
        if support(itemset)>minsupport:
            #Generate rules from each item set m.b. confidence
             for j in range(i):
                rule_to = itemset[j]
                rule_from = [x for x in itemset if x!=itemset[j]]
                confidence=support(itemset)/support(rule_from) 
                if confidence>minconfidence:
                    assocRules.append((rule_from,rule_to))
                    #keep track of the items in the rules for the next iteration
                    for x in itemset:
                        if x not in newSupportItems:
                            newSupportItems.append(x)
    return assocRules, newSupportItems

In [99]:
# Find 2 item sets
minsupport=0.01
minconfidence=0.5
assocRules=[]
newSupportItems=[]

assocRules, supportItems2 =  aprioriIteration(2,supportItems1,assocRules,newSupportItems,minsupport,minconfidence)

In [100]:
len(assocRules)

14

In [101]:
assocRules

[(['46'], '0'),
 (['0'], '46'),
 (['3'], '18'),
 (['3'], '35'),
 (['9'], '4'),
 (['5'], '22'),
 (['44'], '14'),
 (['32'], '16'),
 (['16'], '32'),
 (['35'], '18'),
 (['18'], '35'),
 (['28'], '27'),
 (['27'], '28'),
 (['33'], '42')]

In [102]:
supportItems2

['0',
 '46',
 '3',
 '18',
 '35',
 '4',
 '9',
 '5',
 '22',
 '14',
 '44',
 '16',
 '32',
 '27',
 '28',
 '33',
 '42']

In [103]:
# Find 3 item sets
assocRules, supportItems3 =  aprioriIteration(3,supportItems2,assocRules,newSupportItems,minsupport,minconfidence)

In [104]:
assocRules

[(['46'], '0'),
 (['0'], '46'),
 (['3'], '18'),
 (['3'], '35'),
 (['9'], '4'),
 (['5'], '22'),
 (['44'], '14'),
 (['32'], '16'),
 (['16'], '32'),
 (['35'], '18'),
 (['18'], '35'),
 (['28'], '27'),
 (['27'], '28'),
 (['33'], '42'),
 (['18', '35'], '3'),
 (['3', '35'], '18'),
 (['3', '18'], '35')]

In [105]:
len(assocRules)

17

In [113]:
def ruleMeta(rule):
    rule_from = [itemMap[x] for x in rule[0]]
    return rule_from,itemMap[rule[1]]

In [114]:
[ruleMeta(rule) for rule in assocRules]

[(["'Chocolate' 'Coffee'"], "'Chocolate' 'Cake'"),
 (["'Chocolate' 'Cake'"], "'Chocolate' 'Coffee'"),
 (["'Opera' 'Cake'"], "'Cherry' 'Tart'"),
 (["'Opera' 'Cake'"], "'Apricot' 'Danish'"),
 (["'Napoleon' 'Cake'"], "'Strawberry' 'Cake'"),
 (["'Truffle' 'Cake'"], "'Gongolais' 'Cookie'"),
 (["'Bottled' 'Water'"], "'Berry' 'Tart'"),
 (["'Apricot' 'Croissant'"], "'Blueberry' 'Tart'"),
 (["'Blueberry' 'Tart'"], "'Apricot' 'Croissant'"),
 (["'Apricot' 'Danish'"], "'Cherry' 'Tart'"),
 (["'Cherry' 'Tart'"], "'Apricot' 'Danish'"),
 (["'Tuile' 'Cookie'"], "'Marzipan' 'Cookie'"),
 (["'Marzipan' 'Cookie'"], "'Tuile' 'Cookie'"),
 (["'Cheese' 'Croissant'"], "'Orange' 'Juice'"),
 (["'Cherry' 'Tart'", "'Apricot' 'Danish'"], "'Opera' 'Cake'"),
 (["'Opera' 'Cake'", "'Apricot' 'Danish'"], "'Cherry' 'Tart'"),
 (["'Opera' 'Cake'", "'Cherry' 'Tart'"], "'Apricot' 'Danish'")]